In [1]:
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import numpy as np
import pandas as pd
from dash.dependencies import Input, Output, State
import re

In [2]:
#app=dash.Dash(__name__)

In [3]:
df = pd.read_csv('data sirup update.csv')
df=df.drop(columns=['Unnamed: 0'])

d = {'Barang, ':'Barang',
     'Jasa Lainnya, ':'Jasa Lainnya',
     'Jasa Konsultansi, ':'Jasa Konsultansi',
     'Pekerjaan Konstruksi, ': 'Jasa Konstruksi'}
df = df.replace(d)


tabel_jumlah_tender_tahun2020=df[['nama','waktu']].loc[df['waktu'] != '2021-01-01'].loc[df['waktu'] != '2023-10-01'].loc[df['waktu'] != '2023-04-01'].groupby(['waktu'],as_index=False).count()
tabel_jumlah_tender_tahun2020.rename(columns = {'nama':'Jumlah Tender'}, inplace = True)

tabel1=pd.read_csv('tabel instansi_hps_kategori.csv')
tabel1[['Jasa Konsultansi Badan Usaha', 'Jasa Konsultansi Perorangan','Pekerjaan Konstruksi', 'Pengadaan Barang', 'Jasa Lainnya']].astype(int)

tabel2=pd.read_csv('tabel instansi_metode.csv')

tabel1 = tabel1.sort_values('hps_mean',ascending=False)
tabel1_graph_kategori = tabel1.sort_values('jumlah_tender',ascending=False)

## Rata-rata hps tiap instansi

In [4]:
hps_rata2 = dcc.Graph(id='chart-hps',
              figure={
                  'data':[{'y': tabel1.instansi.head(10),'x':tabel1.hps_mean.head(10), 'type' : 'bar','orientation': 'h', 'name':'first-chart'}],
                  'layout':{'title':'Top 10 Rata-Rata HPS Tiap Instansi'}
              })

## Perbandingan Instansi dg jml tender by kategori

In [5]:
perbandingan_jml_tender = dcc.Graph(id='chart-jumlah-tender',
              figure={
                  'data':[{'x': tabel1_graph_kategori.instansi.head(10),'y':tabel1_graph_kategori.jumlah_tender.head(10), 'type' : 'bar', 'name':'jumlah-tender'},
                         {'x': tabel1_graph_kategori.instansi.head(10),'y':tabel1_graph_kategori['Jasa Konsultansi Badan Usaha'].head(10), 'type' : 'bar', 'name':'jasa-konsultansi-badan-usaha'},
                         {'x': tabel1_graph_kategori.instansi.head(10),'y':tabel1_graph_kategori['Jasa Konsultansi Perorangan'].head(10), 'type' : 'bar', 'name':'jasa-konsultansi-perorangan'},
                         {'x': tabel1_graph_kategori.instansi.head(10),'y':tabel1_graph_kategori['Pekerjaan Konstruksi'].head(10), 'type' : 'bar', 'name':'pekerjaan-konstruksi'},
                         {'x': tabel1_graph_kategori.instansi.head(10),'y':tabel1_graph_kategori['Pengadaan Barang'].head(10), 'type' : 'bar', 'name':'pengadaa-barang'},
                         {'x': tabel1_graph_kategori.instansi.head(10),'y':tabel1_graph_kategori['Jasa Lainnya'].head(10), 'type' : 'bar', 'name':'jasa-lainnya'}],
                  'layout':{'title':'Perbandingan Instansi dengan Jumlah Tender Berdasarkan Kategori'}
              })

## Jumlah tender 2020

In [6]:
jumlah_tender = dcc.Graph(id='chart-2020',
             figure={'data':[go.Scatter(x=tabel_jumlah_tender_tahun2020['waktu'],
                                       y=tabel_jumlah_tender_tahun2020['Jumlah Tender'])
                            ],
                    'layout':go.Layout(title='JUMLAH TENDER 2020',
                                      xaxis={'title':'Bulan Tahun'},
                                      yaxis={'title':'Jumlah Tender'})})

## Tabel tender

In [7]:
tender_kolom=['metode', 'nama', 'instansi', 'lokasi', 'jenis', 'waktu',
        'id', 'pagu', 'satuanKerja']
tabel_tender = dash_table.DataTable(id='table',
                         columns=[{"name": i.title(), "id": i} for i in tender_kolom],
                         data=df.to_dict('records'), page_size=8, style_table={'overflowX': 'scroll'},
                         sort_action='native', selected_columns=[], selected_rows=[],
                         filter_action="native", sort_mode="multi", column_selectable="single",
                         #title = 'TABLE TENDER TAHUN 2020'
                        )

## Cari tender

In [8]:
def list_tender(keyword_tender,pagu,instansi):
    #filter nama tender
    tender = str(keyword_tender)
    matches = tender.split()
    safe_matches = [re.escape(m) for m in matches]
    result = df[df['nama'].astype(str).str.contains('|'.join(safe_matches),case=False)].head(10)
    
    #filter instansi
    instansi = str(instansi)
    matches = instansi.split()
    safe_matches = [re.escape(m) for m in matches]
    result_instansi = result[result['instansi'].astype(str).str.contains('|'.join(safe_matches),case=False)]
    style_instansi={}
    if len(result_instansi) == 0:
        result_instansi = result
        style_instansi = {'color':'red'}
    
    #filter pagu
    style_pagu = {}
    result_pagu = result_instansi[result['pagu']<=int(pagu)].head(10)
    if len(result_pagu) == 0:
        result_pagu = result
        style_pagu = {'color':'red'}
    
    
    div = []
    for index,row in result_pagu.iterrows():
        #print(result)
        div.append(
            html.Div([
                html.Table([
                    html.Td([
                        html.Div([html.A(row['nama'], href='/'+str(row['id']), target='_blank')]),
                        #html.H4(row['nama']),
                        html.Div(row['instansi'],style=style_instansi),
                        html.Div(row['satuanKerja']),

                        html.Div('Pagu : '+str('Rp.{:,.2f}'.format(row['pagu'])) + ' Waktu: '+row['waktu'], style=style_pagu)
                    ])
                ])
            ],style={'overflow':'auto'})
        )
    return div

# Tabel 

In [9]:
print(dcc.__version__) # 0.6.0 or above is required

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Since we're adding callbacks to elements that don't exist in the app.layout,
# Dash will raise an exception to warn us that we might be
# doing something wrong.
# In this case, we're adding the elements through a callback, so we can ignore
# the exception.
app.config.suppress_callback_exceptions = True


1.6.0


# Index

In [10]:
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Layout 1

In [11]:
page_1_layout = html.Div([
    #dcc.Link('Go to Page analisa', href='/analisa', target='_blank'),
    #html.Div([html.A('Go to Page analisa', href='/analisa', target='_blank')]),
    #dcc.Input(id='my-id', value='initial value', type='text'),
    
    html.Div([
            # Filter tender
            html.Div([
                html.Div('Keyword Tender', className='three columns'),
                html.Div(dcc.Input(id='keyword_tender',
                                     value='tender', type='text' ),
                         className='nine columns')
            ]),

            # Filter pagu
            html.Div([
                html.Div('Pagu Maksimal', className='three columns'),
                html.Div(dcc.Input(id='pagu',
                                     value='100000000', type='number' ),
                         className='nine columns')
            ]),

            # Filter instansi
            html.Div([
                html.Div('Instansi', className='three columns'),
                html.Div(dcc.Input(id='instansi',
                                     value='nama instansi', type='text' ),
                         className='nine columns')
            ]),
            
            html.Div(id='my-div', className='six columns')
        ], className='six columns'),
    html.Div([
        hps_rata2,
        jumlah_tender
    ], style={'columnCount': 1}),
    
    
    
    perbandingan_jml_tender,
    #tabel_tender
]) 



In [12]:
@app.callback(
    Output(component_id='my-div', component_property='children'),
    [Input(component_id='keyword_tender', component_property='value'),
    Input(component_id='pagu', component_property='value'),
    Input(component_id='instansi', component_property='value')]
)
def update_output_div(keyword_tender,pagu,instansi):
    return list_tender(keyword_tender,pagu,instansi)

# Histori Tender

In [13]:
histori_tender = pd.read_csv('rekomendasi harga penawaran.csv')
histori_tender = histori_tender.drop(['Unnamed: 0','npwp_pemenang',
                                          'hps','metode pascakualifikasi','metode pemilihan','9','10','11','12','13',
                                         'metode evaluasi','kategori','kategori.1','pagu','hps_detail',
                                         'alamat_pemenang','penawaran_pemenang','preprocessing','rekomendasi_harga_penawaran'],axis=1)
histori_tender = histori_tender[histori_tender['status_tender']=='Tender Sudah Selesai']
histori_tender['nama_tender'] = pd.DataFrame({'nama_tender':[kata.rsplit('<')[0].lower() for kata in histori_tender['nama_tender']]})
def list_histori_tender(instansi):
    #filter nama tender
    ins = str(instansi)
    matches = ins.split()
    safe_matches = [re.escape(m) for m in matches]
    result = histori_tender[histori_tender['instansi'].astype(str).str.contains('|'.join(safe_matches),case=False)].head(10)
    #print(result)
    div = [html.H2('Tender Terdahulu'),html.H4(instansi.values[0].title())]
    for index,row in result.iterrows():
        #print(result)
        div.append(
            html.Div([
                html.Table([
                    html.Td([
                        html.Div([html.A(row['nama_tender'], href='https://lpse.lkpp.go.id/eproc4/lelang/'+str(row['kode'])+'/pengumumanlelang/', target='_blank')]),
                        #html.H4(row['nama']),
                        html.Div(row['kode']),
                        html.Div(row['nama_pemenang']),
                        html.Div('HPS : '+str('Rp.{:,.2f}'.format(row['hps_number']))),
                        html.Div('Penawaran Pemenang : '+str('Rp.{:,.2f}'.format(row['hps_number'])))
                        
                    ])
                ])
            ],style={'overflow':'auto'})
        )
    print('histori')
    return div

# Rekomendasi Tender

In [15]:
rekomendasi_tender = pd.read_csv('data sirup kmeans + lda.csv')
def rekom_tender(kode):
    #print(rekomendasi_tender['id'].values)
    #id tender['kategori_kmeans']
    kategori = rekomendasi_tender.loc[rekomendasi_tender['id']==kode[0]]['kategori_kmeans']
    result = rekomendasi_tender[rekomendasi_tender['kategori_kmeans']==kategori.values[0]].head(10)
    
    div = [html.H2('Rekomendasi Tender')]
    for index,row in result.iterrows():
        #print(result)
        div.append(
            html.Div([
                html.Table([
                    html.Td([
                        html.Div([html.A(row['nama'], href='/'+str(row['id']), target='_blank')]),
                        #html.H4(row['nama']),
                        html.Div(row['instansi']),
                        html.Div(row['satuanKerja']),

                        html.Div('Pagu : '+str('Rp.{:,.2f}'.format(row['pagu'])) + ' Waktu: '+str(row['waktu']))
                    ])
                ])
            ],style={'overflow':'auto'})
        )
    return div

# cari instansi

In [16]:
def list_tender_instansi(instansi):
    
    #filter nama tender
    ins = str(instansi)
    matches = ins.split()
    safe_matches = [re.escape(m) for m in matches]
    result = df[df['instansi'].astype(str).str.contains('|'.join(safe_matches),case=False)].head(10)
    print(instansi)
    div = [html.H2('Tender Terbuka'),]
    for index,row in result.iterrows():
        #print(result)
        div.append(
            html.Div([
                html.Table([
                    html.Td([
                        html.Div([html.A(row['nama'], href='/'+str(row['id']), target='_blank')]),
                        #html.H4(row['nama']),
                        html.Div(row['instansi']),
                        html.Div(row['satuanKerja']),

                        html.Div('Pagu : '+str('Rp.{:,.2f}'.format(row['pagu'])) + ' Waktu: '+row['waktu'])
                    ])
                ])
            ],style={'overflow':'auto'})
        )
    return div

# Layout 2

In [17]:
def detail_tender(selected):
    return [html.H2('Detail Tender'),
        html.Tbody([
            html.Tr([
                html.Td('Nama Tender', colSpan="3"),
                html.Td(selected['nama'].astype(str).str.title())
            ]),
            html.Tr([
                html.Td('Instansi', colSpan="3"),
                html.Td(selected['instansi'].astype(str).str.title())
            ]),
            html.Tr([
                html.Td('Satuan Kerja', colSpan="3"),
                html.Td(selected['satuanKerja'].astype(str).str.title())
            ]),
            html.Tr([
                html.Td('Nilai Pagu', colSpan="3"),
                html.Td(selected['pagu'].astype(str).str.title())
            ]),
            html.Tr([
                html.Td('Lokasi', colSpan="3"),
                html.Td(selected['lokasi'].astype(str).str.title())
            ]),
            html.Tr([
                html.Td('Jenis', colSpan="3"),
                html.Td(selected['jenis'].astype(str).str.title())
            ]),
            html.Tr([
                html.Td('Metode', colSpan="3"),
                html.Td(selected['metode'].astype(str).str.title())
            ])
        ])]


In [18]:
def page_2_layout(id_tender):
    selected = df.loc[df['id']==int(id_tender)].head(1)
    
    
    return html.Div([
    dcc.Store(id='session', storage_type='session'),
    
    html.Table(
        detail_tender(selected)
    ),
    html.Table(
        rekom_tender(selected['id'].values)
    ),
    html.Table(
        list_tender_instansi(selected['instansi'])
    ),
    html.Table(
        list_histori_tender(selected['instansi'])
    ),






    html.H1('Page 2'),

    html.Div(id='page-2-content'),
    html.Br(),
    dcc.Link('Go back to home', href='/')
])

# add a click to the appropriate store.
@app.callback(Output('session', 'data'),
                [Input('{}-button'.format('session'), 'n_clicks')],
                [State('session', 'data')])
def on_click(n_clicks, data):
    if n_clicks is None:
        # prevent the None callbacks is important with the store component.
        # you don't want to update the store for nothing.
        raise PreventUpdate

    # Give a default data dict with 0 clicks if there's no data.
    data = data or {'clicks': 0}

    data['clicks'] = data['clicks'] + 1
    return data

# output the stored clicks in the table cell.
@app.callback(Output('{}-clicks'.format('session'), 'children'),
                # Since we use the data prop in an output,
                # we cannot get the initial data on load with the data prop.
                # To counter this, you can use the modified_timestamp
                # as Input and the data as State.
                # This limitation is due to the initial None callbacks
                # https://github.com/plotly/dash-renderer/pull/81
                [Input('session', 'modified_timestamp')],
                [State('session', 'data')])
def on_data(ts, data):
    if ts is None:
        raise PreventUpdate

    data = data or {}

    return data.get('clicks', 0)

@app.callback(dash.dependencies.Output('page-2-content', 'children'),
              [dash.dependencies.Input('page-2-radios', 'value')])
def page_2_radios(value):
    return 'You have selected "{}"'.format(value)

# Route

In [19]:
# Update the index
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/':
        #return layout_grid
        return page_1_layout
    else:
        id_tender = re.findall("\d+", pathname)
        if len(id_tender)>0:
            return page_2_layout(id_tender[0])
        else:
            return page_1_layout
    # You could also return a 404 "URL not found" page here

In [20]:
if __name__=='__main__':
    app.run_server(port=4050)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4050/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Dec/2019 20:47:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:24] "GET /_favicon.ico?v=1.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2019 20:47:33] "POST /_dash-update-component HT

# app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])


index_page = html.Div([
    dcc.Link('Go to Page 1', href='/page-1'),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2'),
])

page_1_layout = html.Div([
    html.H1('Page 1'),
    dcc.Dropdown(
        id='page-1-dropdown',
        options=[{'label': i, 'value': i} for i in ['LA', 'NYC', 'MTL']],
        value='LA'
    ),
    html.Div(id='page-1-content'),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2'),
    html.Br(),
    dcc.Link('Go back to home', href='/'),
])

@app.callback(dash.dependencies.Output('page-1-content', 'children'),
              [dash.dependencies.Input('page-1-dropdown', 'value')])
def page_1_dropdown(value):
    return 'You have selected "{}"'.format(value)





# Update the index
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page
    # You could also return a 404 "URL not found" page here


if __name__ == '__main__':
    app.run_server(debug=True)

Dash app with multiple pages

In [21]:
df.loc[df['id']==22319671].head(1)
df['jenis'].unique()
d = {'Barang, ':'Barang',
     'Jasa Lainnya, ':'Jasa Lainnya',
     'Jasa Konsultansi, ':'Jasa Konsultansi',
     'Pekerjaan Konstruksi, ': 'Jasa Konstruksi'}
df = df.replace(d)
df['jenis'].unique()

array(['Barang', 'Jasa Konstruksi', 'Jasa Lainnya', 'Jasa Konsultansi',
       'Pekerjaan Konstruksi'], dtype=object)